# CNN Sentence Classification (with Theano code)

## 1. Intro

本篇博客来细说CNN在NLP中的一大应用————句子分类。

传统的句子分类器一般使用SVM和Naive Bayes。传统方法使用的文本表示方法大多是“词袋模型”。即只考虑文本中词的出现的频率，不考虑词的序列信息。传统方法也可以强行使用N-gram的方法，但是这样会带来稀疏问题，意义不大。

**CNN（卷积神经网络）**，虽然出身于图像处理，但是它的思路，给我们提供了在NLP应用上的参考。**“卷积”**这个术语本身来自于信号处理，它的物理意义可以参考[知乎上关于“复利”的回答](https://www.zhihu.com/question/22298352?rf=21686447)，或者参考[colah大神的博客](http://colah.github.io/posts/2014-07-Understanding-Convolutions/)。简单地说就是一系列的输入信号进来之后，系统也会有一系列的输出。但是并不是某一时刻的输出只对应该时刻的输入，而是根据系统自身的特征，**每一个时刻的输出，都和之前的输入相关。**那么如果文本是一些列输入，我们当然希望考虑词和词的序列特征，比如“Tom 的 手机 ”，使用卷积，系统就会知道“手机是tom”的，而不是仅仅是一个“手机”。

或者更直观地理解，在CNN模型中，卷积就是拿**kernel**在图像上到处移动，每移动一次提取一次特征，组成feature map， 这个提取特征的过程，就是卷积。

接下来，我们看看Yoon Kim的paper：[Convolutional Neural Networks for Sentence Classification](http://arxiv.org/abs/1408.5882) (EMNLP 2014)


## 2. 论文介绍

Yoon Kim 自己画的结构图：

![](https://raw.githubusercontent.com/applenob/nlp_projects/master/resources/cnn.png)

具体结构介绍：

**1.输入层**

可以把输入层理解成把一句话转化成了一个二维的图像：每一排是一个词的word2vec向量，纵向是这句话的每个词按序排列。输入数据的size，也就是图像的size，**n×k**，n代表训练数据中最长的句子的词个数，这里是64（不够64个词的句子采用zero padding），k是embbeding的维度，这里是300。所谓的static和non-static的chanel解释如下，[来自](http://www.jeyzhang.com/cnn-apply-on-modelling-sentence.html)：

- CNN-rand: 所有的word vector都是随机初始化的，同时当做训练过程中优化的参数；
- CNN-static: 所有的word vector直接使用无监督学习即Google的Word2Vector工具(COW模型)得到的结果，并且是固定不变的；
- CNN-non-static: 所有的word vector直接使用无监督学习即Google的Word2Vector工具(COW模型)得到的结果，但是会在训练过程中被Fine tuned；
- CNN-multichannel: CNN-static和CNN-non-static的混合版本，即两种类型的输入；

从输入层还可以看出kernel的size。很明显kernel的高(h)会有不同的值，图上有的是2，有的是3。这很容易理解，不同的kernel想获取不同范围内词的关系；和图像不同的是，**nlp中的cnn的kernel的宽(w)一般都是图像的宽**，也就是word2vec的维度，这也可以理解，因为我们需要获得的是纵向的差异信息，也就是不同范围的词出现会带来什么信息。

**2.卷积层**

由于kernel的特殊形状，因此卷积后的feature map是一个宽度是1的长条。

**3.池化层**

这里使用是MaxPooling，并且一个feature map只选一个最大值留下。这被认为是按照这个kernel卷积后的最重要的特征。

**4.全连接层**

这里的全连接层是带dropout的全连接层和softmax。